In [1]:
import requests
import json
import pandas as pd

In [2]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',

}

url = 'https://eleapi.interno.gov.it/siel/PX/getentiR/DE/20241117'

response = requests.get(url, headers=headers)
data = response.json()

In [3]:
enti_data = []

for ente in data['enti']:
  ente_info = {
    'cod' : ente['cod'],
    'desc' : ente['desc'],
    'tipo': ente['tipo'],
    'tipo_comune': ente['tipo_comune'],
    'dt_agg': ente['dt_agg'],
    'cod_ele': ente['cod_ele']
  }

  enti_data.append(ente_info)

enti_df = pd.DataFrame(enti_data)

enti_df.head(2)

,cod,desc,tipo,tipo_comune,dt_agg,cod_ele
0,080000000000,EMILIA-ROMAGNA,RE,None,None,7
1,080130000000,BOLOGNA,CR,None,None,7


In [4]:
df_pr = enti_df[enti_df['tipo'] == 'PR']
province_mapping = df_pr.set_index(df_pr['cod'].str[2:5])['desc'].to_dict()

def map_provincia(cod):
    provincia_cod = cod[2:5]  # Extract the province code (3rd to 5th digits)
    return province_mapping.get(provincia_cod, 'Unknown')

df_re = enti_df[enti_df['tipo'] == 'RE']
regione_mapping = df_re.set_index(df_re['cod'].str[:2])['desc'].to_dict()

def map_regione(cod):
    regione_cod = cod[:2]  # Extract the province code (3rd to 5th digits)
    return regione_mapping.get(regione_cod, 'Unknown')

In [5]:
enti_df['cod_regione'] = enti_df['cod'].str[:2]
enti_df['regione'] = enti_df['cod'].apply(map_regione)

enti_df['cod_provincia'] = enti_df['cod'].str[2:5]
enti_df['provincia'] = enti_df['cod'].apply(map_provincia)

enti_df['cod_provincia_2'] = enti_df['cod'].str[5:8]
enti_df['provincia'] = enti_df['cod'].apply(map_provincia)

enti_df['cod_comune'] = enti_df['cod'].str.slice(-4)

enti_df.head(2)

,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune
0,080000000000,EMILIA-ROMAGNA,RE,None,None,7,08,EMILIA-ROMAGNA,000,Unknown,000,0000
1,080130000000,BOLOGNA,CR,None,None,7,08,EMILIA-ROMAGNA,013,BOLOGNA,000,0000


In [ ]:
enti_df.to_csv('../output/codici_umbria_er_20241117.csv', index=False, encoding='UTF-8-sig')